In [19]:
from model_utils import *
import spacy
from spacy import tokenizer
from spacy import displacy
import numpy as np

nlp = spacy.load("en_core_web_sm")

import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [3]:
ner_model, preproc = load_model()

404 Client Error: Not Found for url: https://huggingface.co/emilyalsentzer/Bio_ClinicalBERT/resolve/main/tf_model.h5


Model Loaded from disk


In [4]:
user_input = "Electrocardiogram findings reveal the patient suffered a myocardial infarction which was treated with ACE inhibitors"

In [51]:
def get_predictions_test(loaded_model, features, text):
    predictor = ktrain.get_predictor(loaded_model, features)
    return predictor.predict(text, return_proba=True)

In [52]:
lst = get_predictions_test(ner_model, preproc, user_input)

In [53]:
text = " ".join([t[0] for t in lst])
print("text joined with single space:\n%s\n\n" % (text))

text joined with single space:
electrocardiogram findings reveal the patient suffered a myocardial infarction which was treated with ace inhibitors




In [54]:
total = 0
lst2 = []
for token in lst:
    start = total
    end = total+len(token[0])
    label = token[1]
    prob = str(round(np.prod(token[2])*100,1))+'%'
    total += 1 + len(token[0])
    if label == 'O': continue
    lst2.append({'start':start, 'end':end, 'label':label.upper()})

lst2

[{'start': 55, 'end': 56, 'label': 'PROBLEM'},
 {'start': 57, 'end': 67, 'label': 'PROBLEM'},
 {'start': 68, 'end': 78, 'label': 'PROBLEM'}]

In [55]:
def displacy_format_old(lst):
    total = 0
    lst2 = []
    for token in lst:
        start = total
        end = total + len(token[0])
        label = token[1]
        prob = str(round(np.prod(token[2]) * 100, 1)) + '%'
        total += 1 + len(token[0])
        if label == 'O': continue
        lst2.append({'start': start, 'end': end, 'label': label.upper() + " " + str(prob)})
    return lst2

In [ ]:
def displacy_format(lst):
    total = 0
    lst2 = []
    for token in lst:
        start = total
        end = total + len(token[0])
        label = token[1]
        prob = str(round(np.prod(token[2]) * 100, 1)) + '%'
        total += 1 + len(token[0])
        if label == 'O': continue
        lst2.append({'start': start, 'end': end, 'label': label.upper()})
    return lst2

In [56]:
ex = {
    "text": "Electrocardiogram findings reveal the patient suffered a myocardial infarction which was treated with ACE inhibitors.",
    "ents": [{"start": 0, "end": 17, "label": "TEST"}, {"start": 54, "end": 78, "label": "PROBLEM"}, {"start":101, "end":116, "label":"TEST"}]
}


In [57]:
dct = {
    'text': user_input,
    'ents': lst2
}

In [ ]:
# +" "+str(prob)

In [58]:
colors = {
    "PROBLEM": "linear-gradient(90deg, #aa9cfc, #fc9ce7)",
    "TEST": "linear-gradient(90deg, #ffafbd, #ffc3a0)",
    "TREATMENT": "linear-gradient(90deg, #02aab0, #00cdac)"
}
options = {"ents": ["PROBLEM", "TEST", "TREATMENT"], "colors": colors}

html = displacy.render(dct, style="ent", manual=True, options=options)

In [64]:
def get_predictions_test2(loaded_model, features, text):
    predictor = ktrain.get_predictor(loaded_model, features)
    return predictor.predict(text, merge_tokens=True)

In [65]:
get_predictions_test2(ner_model, preproc, user_input)

[]

In [31]:
print('converted to character indices:\n%s\n\n'% (lst2))
print('checking if character indicies are correct...')

for d in lst2:
    print(text[d['start']:d['end']])

converted to character indices:
[{'start': 55, 'end': 56, 'label': 'PROBLEM 100.0%'}, {'start': 57, 'end': 67, 'label': 'PROBLEM 100.0%'}, {'start': 68, 'end': 78, 'label': 'PROBLEM 100.0%'}]


checking if character indicies are correct...
a
myocardial
infarction


In [7]:
df = pd.DataFrame(get_predictions_test(ner_model, preproc, user_input))
df.columns = ["token", "label", "prob"]
df

,token,label,prob
0,electrocardiogram,O,0.999989
1,findings,O,0.999992
2,reveal,O,0.999997
3,the,O,0.999873
4,patient,O,0.999674
5,suffered,O,0.999938
6,a,problem,0.999656
7,myocardial,problem,0.999938
8,infarction,problem,0.999814
9,which,O,0.999987


In [30]:
df.groupby('label')["token"].apply(list)

label
O          [electrocardiogram, findings, reveal, the, pat...
problem                          [a, myocardial, infarction]
Name: token, dtype: object

In [33]:
colors = {
    "PROBLEM": "linear-gradient(90deg, #aa9cfc, #fc9ce7)",
    "TEST": "linear-gradient(90deg, #ffafbd, #ffc3a0)",
    "TREATMENT": "linear-gradient(90deg, #02aab0, #00cdac)"
}
options = {"ents": ["PROBLEM", "TEST", "TREATMENT"], "colors": colors}

ex = {
    "text": "Electrocardiogram findings reveal the patient suffered a myocardial infarction which was treated with ACE inhibitors.",
    "ents": [{"start": 0, "end": 17, "label": "TEST"}, {"start": 54, "end": 78, "label": "PROBLEM"}, {"start":101, "end":116, "label":"TEST"}]
}
html = displacy.render(ex, style="ent", manual=True, options=options)

KeyError: 0

In [35]:
len("Electrocardiogram")

17

In [43]:
df.token.apply(lambda x: sample_doc.find(" "+x))

0      -1
1      17
2      26
3      33
4      37
5      45
6      54
7      56
8      67
9      78
10     84
11     88
12     96
13    101
14    105
Name: token, dtype: int64

In [31]:
sample_doc = " ".join(user_input.strip().split("\n"))

'here is some text about a she had a lot of issues with a hernia'

In [ ]:
## displacy format:

# {
#     "text": "But Google is starting from behind.",
#     "ents": [{"start": 4, "end": 10, "label": "ORG"}],
#     "title": None
# }

In [26]:
displacy.render(nlp(user_input), style="ent", jupyter=True, manual=True)

TypeError: render() got an unexpected keyword argument 'render'